이름: 나영훈

학번: 20185124

# Neural networks with PyTorch

Pytorch의 `nn.module`을 활용하여 만드는 유용한 방법을 학습합니다.

<div style="text-align:center"><img src='https://drive.google.com/uc?export=download&id=1J2SeiPpVJs1-ML2BdLrcxkGGmHpRxIVE' width="250" height="200"> 

### Lego block coding! </div>

In [1]:
import torch
import numpy as np

from torch import nn
import torch.nn.functional as F
#ReLU, sigmoid .. ect 를 사용하기 위해 import torch.nn.functional as F 를 부름 (nn 사용도 가능)
from collections import OrderedDict
#OrderedDict : 순서가 있는 Dictionary

torch.manual_seed(0)

`nn.Linear`: $Z^{[\ell]} = A^{[\ell-1]}W^T+b$
연산.

해당 layer의 

- 입력 차원 `n_input=30`
- 출력 차원 `n_output=60`


In [2]:
# Example of nn.linear
linear_layer1 = nn.Linear(30, 60)

In [3]:
A = torch.randn(60, 30) 
linear_layer1(A).shape

torch.Size([60, 60])

Why 60, 60 -> (60,30) x (30,60) = (60, 60)

How to get the weights and bias of each `nn.Linear`

In [4]:
# Example of weights
linear_layer1.weight.data = torch.ones_like(linear_layer1.weight)

In [5]:
linear_layer1.weight

Parameter containing:
tensor([[1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        ...,
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.],
        [1., 1., 1.,  ..., 1., 1., 1.]], requires_grad=True)

In [6]:
linear_layer1.bias.shape
#bias의 shape은 항상 1차원이다. 또한 bias의 크기는 output과 같다.

torch.Size([60])

### NN example

- input units: 20
- hidden layer: 30, 40
- output units: 3
- activation function: ReLU
- output layer: No activation



In [7]:
# Simple NN construction

class FCN(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.lin1 = nn.Linear(20, 30)
        self.lin2 = nn.Linear(30, 40)
        self.lin3 = nn.Linear(40, 3)
        self.relu = nn.ReLU(True)
        
    def forward(self, x):
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        x = self.relu(x)
        x = self.lin3(x)
        return x
    


In [8]:
Xtrain = torch.randn(60, 20)

model = FCN()
# print(model(Xtrain)) # print(model.forward(FCN()) 여기서 method의 이름이 forward이면 생략 가능함.
print(model(Xtrain).shape)

torch.Size([60, 3])


In [9]:
model

FCN(
  (lin1): Linear(in_features=20, out_features=30, bias=True)
  (lin2): Linear(in_features=30, out_features=40, bias=True)
  (lin3): Linear(in_features=40, out_features=3, bias=True)
  (relu): ReLU(inplace=True)
)

In [10]:
#파라메타 확인하기.
Xtrain = torch.randn(60, 20)

model = FCN()

# model.lin1.weight.data = torch.ones(30,20, requires_grad = True)
model.lin1.bias.data = torch.ones(60)
model.lin1.bias.shape

torch.Size([60])

#### My Neural Network

In [11]:
# My Custom Model

class MyNN(nn.Module):
    def __init__(self):
        # print("hello")
        
        super().__init__() #super(MyNN, self).__init__() 
        #super 클래스의 __init__ 을 사용할 수 있게 초기화 시켜줌.
        
        self.lin1 = nn.Linear(20,30)
        self.lin2 = nn.Linear(30,40)
        self.lin3 = nn.Linear(40,3)
        
        self.relu = nn.ReLU(True)        

    def forward(self, x): #forward pass를 할 때 사용하는 연산 정의
        x = self.lin1(x)
        x = self.relu(x)
        x = self.lin2(x)
        x = self.relu(x)
        x = self.lin3(x)
        
        return x
        #마지막 layer는 Not use Activation Function
        

In [12]:
model = MyNN()
model

MyNN(
  (lin1): Linear(in_features=20, out_features=30, bias=True)
  (lin2): Linear(in_features=30, out_features=40, bias=True)
  (lin3): Linear(in_features=40, out_features=3, bias=True)
  (relu): ReLU(inplace=True)
)

In [13]:
print(model.lin1)
print(model.lin2)

Linear(in_features=20, out_features=30, bias=True)
Linear(in_features=30, out_features=40, bias=True)


### 다시 수업으로 Back

In [14]:
# Example of parameters() in models

#상속 받을 때 super 초기화 후 forward에서 이미 계산이 다 됨.
param_iterator = model.parameters()
param_iterator

for param in param_iterator:
    # param.requires_grad = False
    # print(param.requires_grad)
    print(param.requires_grad)


True
True
True
True
True
True


## nn.Sequential() 을 사용하여 Layer 구성하기

In [15]:
# nn.Sequential() example

class FCN_seq(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(
                    nn.Linear(20, 30),
                    nn.ReLU(True),
                    nn.Linear(30, 40),
                    nn.ReLU(True),
                    nn.Linear(40, 3)
        )
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [16]:
model_seq = FCN_seq()
Xtrain.shape
model_seq

FCN_seq(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [17]:
# model_seq(Xtrain) 

In [18]:
model_seq.fc[0]

Linear(in_features=20, out_features=30, bias=True)

In [19]:
model_seq.fc[1]

ReLU(inplace=True)

In [20]:
model_seq.fc[2]

Linear(in_features=30, out_features=40, bias=True)

In [21]:
class FCN_seq_v2(nn.Module):
    def __init__(self):
        super().__init__()
        

        
        temp = self.fcn_block(20, 30)+self.fcn_block(30, 40)+[nn.Linear(40,1)] 
        # fan_block이라는 함수를 정의함. nn.Linear 와 nn.ReLU를 세트로 가짐. -> 간략화 가능.
        
        self.fc = nn.Sequential(*temp)
        # *temp는 풀어서 나열.
        
        
    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim), 
                             nn.ReLU(True)] #리스트로 만들어서 return
        
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [22]:
model_seq_v2 = FCN_seq_v2()
model_seq_v2

FCN_seq_v2(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=1, bias=True)
  )
)

In [23]:
class FCN_final(nn.Module):
    
                      #  20     30,40      3 
    def __init__(self, in_dim, hlayer, out_dim):
        super().__init__()
        
        l_list = self.fcn_block(in_dim, hlayer[0]) #fan_block은 반환형이 list이므로 list를 안씌워줘도 됨.
        
        for l1, l2 in zip(hlayer[:-1], hlayer[1:]):
            l_list = l_list + self.fcn_block(l1, l2)
        
        l_list = l_list + [nn.Linear(hlayer[-1], out_dim)] #Linear는 List가 아니라서 list를 씌워줘야 함.
        
        self.fc = nn.Sequential(*l_list)
        
    # fcn_block 이라는 사용자 정의 함수 만들기.     
    def fcn_block(self, in_dim, out_dim):
        return [nn.Linear(in_dim, out_dim),
                             nn.ReLU(True)]
        
        
    def forward(self, x):
        return self.fc(x)
        
        

In [24]:
# ZIP은 Iterator를 반환해줌.
# 순서대로 리스트의 원소를 묶어서 반환 가능.

# hlayer1 = [30, 40, 50, 60]
# hlayer2 = [20, 80, 99, 100]

# for i, j in zip(hlayer1, hlayer2):
#   print(i,j)

In [25]:
hlayer = [30, 40]
in_dim = 20
out_dim= 3

myfcn_final = FCN_final(in_dim, hlayer, out_dim)

In [28]:
hlayer = [30,40,50,60]
# hlayer[:-1] = [30,40,50]
# hlayer[1:]  = [40,50,60]

for i,j in zip(hlayer[:-1], hlayer[1:]):
    print(i,j)

30 40
40 50
50 60


In [29]:
myfcn_final

FCN_final(
  (fc): Sequential(
    (0): Linear(in_features=20, out_features=30, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=30, out_features=40, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [30]:
# Ordered dict example
# nn.Sequential() example

class FCN_seq_ordered_dic(nn.Module):
    def __init__(self):
        super().__init__()
        
        self.fc = nn.Sequential(OrderedDict([('lin1', nn.Linear(20, 30)), # 이름 짓고 싶을 때 순서가 있는 OrderedDict 사용, 내용물은 list여야함.
                      ('relu1', nn.ReLU(True)),
                      ('lin2', nn.Linear(30, 40)),
                      ('relu2',nn.ReLU(True)),
                      ('lin3', nn.Linear(40, 3))
                                            ])
        )
        
    def forward(self, x):
        return self.fc(x)
        
        
        
# self.lin1 = nn.Linear(20, 30)
# self.lin2 = nn.Linear(30, 40)
# self.lin3 = nn.Linear(40, 3)
# self.relu = nn.ReLU(True)

In [31]:
# 이름만 불러도 되는 엄청난 친구... 대박.. 외워두기!!!
# 남들에게 배포할 떄 이름 붙이기 때문에...! 
FCN_seq_ordered_dic()

FCN_seq_ordered_dic(
  (fc): Sequential(
    (lin1): Linear(in_features=20, out_features=30, bias=True)
    (relu1): ReLU(inplace=True)
    (lin2): Linear(in_features=30, out_features=40, bias=True)
    (relu2): ReLU(inplace=True)
    (lin3): Linear(in_features=40, out_features=3, bias=True)
  )
)

In [32]:
# ModuleList(), ModuleDict()

In [36]:
# state_dict() example

# 내 모델, 모델에 대한 이름, 모델에 해당하는 파라메타, bias , Layer 별 Weight and bias
model.state_dict()

OrderedDict([('lin1.weight',
              tensor([[ 0.1894, -0.1852, -0.1165,  0.0401, -0.2231,  0.0509, -0.0190, -0.2097,
                       -0.0556, -0.1636,  0.0216,  0.1648,  0.2128, -0.2091,  0.0797, -0.1721,
                        0.0795,  0.1231, -0.2130, -0.1614],
                      [-0.2005, -0.0210,  0.0819, -0.1640, -0.0145, -0.0778,  0.2029, -0.2050,
                       -0.1241, -0.0021, -0.1743,  0.1721, -0.2054, -0.0721, -0.0935, -0.0020,
                       -0.1062,  0.1605, -0.1925, -0.1179],
                      [-0.1023, -0.2137, -0.2218,  0.0789,  0.2106, -0.0342, -0.0868,  0.0978,
                        0.1318, -0.2007, -0.1068,  0.1890,  0.0995, -0.2076,  0.0042, -0.1693,
                       -0.1730, -0.1111,  0.2219, -0.1289],
                      [ 0.1537, -0.2029, -0.0154,  0.2022, -0.0502, -0.0771, -0.0989, -0.1616,
                        0.0717, -0.1630, -0.1422,  0.1942,  0.1664, -0.1169,  0.0047, -0.0601,
                        0.1702,

# Problem Setup

<div style="text-align:center"> <img src='https://drive.google.com/uc?export=download&id=1FRhniwGeeutBSJQRdW6GzshMfDrPz7oJ' width="250" height="200"> </div>
    
Build a Fully connected neural network with

- 3 layers
- 마지막 layer의 unit 수는 `1` 
  - 마지막 layer의 activation은 없음 (linear layer)
- Data feature 수는 `100`

- input unit 수는 data 크기를 보고 맞추세요
- hidden layer의 unit 수는 `[80, 50]`
  - hidden layer의 activation 함수는 ReLU

- model class 명 `myFCN`
  - instance 명 `my_model` 생성
  - `my_model` 출력

## Problem 1

problem setup에서 구성한 neural network을 `nn.Sequential`을 활용하여 생성하세요

In [37]:
## 사용할 data 
batch_size = 30
num_feature = 100

X_train = torch.randn(batch_size, num_feature)

In [38]:
X_train.shape

torch.Size([30, 100])

In [39]:
# Problem 1 코딩 (매 줄마다 주석 필수 )

class myFCN(nn.Module):                     # n .Module을 상속받은 myFCN 이라는 이름의 class를 정의합니다. super class : nn.Module, sub class : myFCN
    def __init__(self):                     # __init__(self) init는 초기화 함수입니다.
        super().__init__()                  # super().__init__() 를 통해 super class인 nn.Module을 초기화 하여 사용 가능하게 만듭니다.

        self.fc = nn.Sequential(           # self.fc는 nn.Sequential로 만sd a들어진 Neural Network 입니다. fc는 fully connnected Layer
                   nn.Linear(100, 80),     # Hidden Layer의 unit수가 [80, 50] 이고 Input data는 100 이므로 첫번째 Layer는 nn.Linear(100,80)
                   nn.ReLU(True),          # 마지막 Output Layer를 제외하고는 Activation Function으로 ReLU를 넣는다. 
                                    # True를 쓰게 되면 메모리 위치를 두고 그 위치에서 연산 수행, inplace 연산. (default = False), 메모리 절약
                   nn.Linear(80,50),       # 이 전 Layer의 shape는 (100,80)이므로 이번 Layer의 Input은 80이여야 한다. 또한 2번째 Hidden Layer의 Unit이 50이므로 (80,50)
                   nn.ReLU(True),          # 마지막 Output Layer가 아니므로 Activation Function으로 ReLU 사용.
                   nn.Linear(50,1)         # 이 전 Hidden Layer의 unit이 50, 마지막 Layer의 unit수는 1이므로 마지막 Layer의 shape는 (50,1)
        )
          
    def forward(self, x):                  # forward pass 진행.
        return self.fc(x)                  # fully connected layer에 입력 데이터인 X가 들어감. x의 shape는 (n, 100)이 되야함.

In [40]:
my_model = myFCN() # myFCN 클래스로 만들어진 fully connected Neural Network를 를 my_model 이라는 변수에 집어 넣습니다.
my_model           # my_model을 출력합니다. 출력 결과는 myFCN 클래스에서 직접 만든 Neural Network 형태가 됩니다.

myFCN(
  (fc): Sequential(
    (0): Linear(in_features=100, out_features=80, bias=True)
    (1): ReLU(inplace=True)
    (2): Linear(in_features=80, out_features=50, bias=True)
    (3): ReLU(inplace=True)
    (4): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [49]:
mm

tensor([[-0.1542, -0.0652, -0.0659, -0.1898, -0.1814, -0.1381],
        [-0.1315, -0.1371, -0.0740, -0.0801, -0.0218, -0.1093],
        [-0.0469, -0.2036, -0.0678, -0.1444, -0.0597, -0.1447],
        [-0.1343, -0.0840, -0.1385, -0.1610, -0.0923, -0.1432],
        [-0.0286, -0.0775, -0.1954, -0.1301, -0.1171, -0.0879]],
       grad_fn=<ReshapeAliasBackward0>)

In [50]:
mm = my_model(X_train).reshape(5,6)
mm[1,0]

tensor(-0.1315, grad_fn=<SelectBackward0>)

In [41]:
my_model(X_train)

tensor([[-0.1542],
        [-0.0652],
        [-0.0659],
        [-0.1898],
        [-0.1814],
        [-0.1381],
        [-0.1315],
        [-0.1371],
        [-0.0740],
        [-0.0801],
        [-0.0218],
        [-0.1093],
        [-0.0469],
        [-0.2036],
        [-0.0678],
        [-0.1444],
        [-0.0597],
        [-0.1447],
        [-0.1343],
        [-0.0840],
        [-0.1385],
        [-0.1610],
        [-0.0923],
        [-0.1432],
        [-0.0286],
        [-0.0775],
        [-0.1954],
        [-0.1301],
        [-0.1171],
        [-0.0879]], grad_fn=<AddmmBackward0>)

## Problem 2

problem setup에서 구성한 neural network을 `OrderedDict`을 활용하여 생성하세요
- 각 layer의 이름을 주고 생성하세요

In [289]:
# 답작성

In [295]:
class myFCN(nn.Module): # myFCN2라는 이름의 클래스를 만듭니다. nn.Module을 상속받아 nn.Module의 여러 속성들을 사용
    def __init__(self): # __init__ 을 통해 초기화 진행.
        super().__init__() # super class 즉 nn.Module을 사용하기 위해 초기화를 진행합니다. __init__()를 통해 초기화 진행.
    
        self.fc = nn.Sequential(OrderedDict([ #OrderDict를 사용하여 모델의 Layer 쌓기, OrderDict를 사용하면 각 Layer에 이름을 줄 수 있음.
                        ('lin1', nn.Linear(100,80)), # lin1 이라는 이름의 Layer는 입력 Unit이 100이 되며 다음 Hidden Layer의 Unit이 80이므로 (100,80)이 됩니다.
                        ('ReLU1', nn.ReLU(True)),    # ReLU1 이라는 이름의 Layer는 lin1 layer에 activation 함수로 ReLU 를 적용합니다. (마지막 레이어가 아니므로 Activation Function 사용)
                        ('lin2', nn.Linear(80,50)),  # lin2 이라는 이름의 layer는 입력 Unit이 80, 다음층의 Unit이 50이므로 (80,50) 이 됨.
                        ('ReLU2', nn.ReLU(True)),    # ReLU2 이라는 이름의 Layer는 lin2 layer에 activation 함수로 ReLU 를 적용.
                        ('lin3', nn.Linear(50,1))    # lin3이라는 이름의 Layer는 마지막 Layer, 입력 Unit의 수가 50, 출력 Unit 수가 1이므로 (50,1) 이 됨. 
                                                     #  마지막 Layer는 Activation Function 사용 안하기 때문에 이 상태로 종료.
                        ]))
                               
    
    def forward(self, x):  # forward pass 진행.
        return self.fc(x)  # fully connected layer에 입력 데이터인 X가 들어감. x의 shape는 (n, 100)이 되야함.
        

In [296]:
# my_model2 = myFCN2()
# my_model2

myFCN()

myFCN(
  (fc): Sequential(
    (lin1): Linear(in_features=100, out_features=80, bias=True)
    (ReLU1): ReLU(inplace=True)
    (lin2): Linear(in_features=80, out_features=50, bias=True)
    (ReLU2): ReLU(inplace=True)
    (lin3): Linear(in_features=50, out_features=1, bias=True)
  )
)

In [292]:
my_model2(X_train)

tensor([[-2.1076e-02],
        [ 1.2121e-01],
        [ 1.7963e-01],
        [ 3.0297e-02],
        [-1.2761e-02],
        [-1.2974e-01],
        [ 8.2657e-02],
        [ 6.4025e-03],
        [ 5.2668e-02],
        [ 5.0146e-02],
        [ 9.9481e-02],
        [ 3.7431e-03],
        [ 7.5810e-02],
        [-7.7921e-02],
        [ 7.1664e-03],
        [ 1.8505e-02],
        [ 1.2403e-01],
        [ 4.1419e-02],
        [-4.9096e-02],
        [ 2.3271e-02],
        [ 2.7100e-02],
        [-2.9133e-02],
        [-6.8623e-02],
        [ 2.0562e-02],
        [ 3.1696e-02],
        [-1.0196e-02],
        [ 2.0711e-02],
        [-1.4658e-04],
        [ 4.7573e-02],
        [-4.9960e-02]], grad_fn=<AddmmBackward0>)